<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/5_Aprendizaje_supervisado/2_Taller_Regresion_Polinomica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# **Taller: Regresión Polinómica, Subajuste, Sobreajuste y Regularización**

**IMPORTANTE**: Guarda una copia de este notebook en tu Google Drive o computador.

**Taller en parejas**

**Nombres estudiantes:**

- 
-

**Forma de entrega**

Jupyter Notebook publicado en su cuenta de Github con el nombre “Taller_Reg_Polin_apellidos_estudiantes.ipynb”.

**Plazo de entrega**

30 de octubre, máximo a las 11:59 p.m., debes enviar link del notebook al correo lina.castro6@uexternado.edu.co, de lo contrario, no será tenido en cuenta.

**Instrucciones Generales**

Completa el código en las celdas marcadas con `### TU CÓDIGO AQUÍ ###`. Puedes añadir más celdas si lo requieres.

## **Situación**

Una importante firma de inversión inmobiliaria te ha contratado como consultor de ciencia de datos. Su proceso actual de valoración de propiedades es lento y se basa en la intuición de unos pocos expertos. Quieren que desarrolles un modelo de machine learning para predecir el precio de venta de las viviendas (`SalePrice`) de forma más precisa y sistemática.

Te han entregado el dataset "Ames Housing", que contiene una gran cantidad de información sobre viviendas vendidas recientemente. Tu tarea es construir el mejor modelo posible, pero más importante aún, justificar por qué tu modelo es robusto y fiable, explicando cómo has manejado la complejidad y el riesgo de sobreajuste.

### **Ejercicio 1: Carga y Preparación Inicial de los Datos**

Primero, carguemos las librerías necesarias y el dataset. Nos enfocaremos en un subconjunto de las variables numéricas para mantener el taller manejable, pero el principio se aplica a todo el dataset.

In [1]:
# Importa las librerías necesarias
### TU CÓDIGO AQUÍ ###
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Importación de herramientas de scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

Mejorar visualización de dataframes y gráficos

In [2]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

# Configuraciones para una mejor visualización
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

Cargar el dataset

In [3]:
# Usamos el dataset Ames Housing desde su fuente original (requiere internet)
url = 'http://jse.amstat.org/v19n3/decock/AmesHousing.txt'
df = pd.read_csv(url, sep='\t')
df.head()

Order        PID  MS SubClass MS Zoning  Lot Frontage  Lot Area Street  \
0      1  526301100           20        RL        141.00     31770   Pave   
1      2  526350040           20        RH         80.00     11622   Pave   
2      3  526351010           20        RL         81.00     14267   Pave   
3      4  526353030           20        RL         93.00     11160   Pave   
4      5  527105010           60        RL         74.00     13830   Pave   

  Alley Lot Shape Land Contour Utilities Lot Config Land Slope Neighborhood  \
0   NaN       IR1          Lvl    AllPub     Corner        Gtl        NAmes   
1   NaN       Reg          Lvl    AllPub     Inside        Gtl        NAmes   
2   NaN       IR1          Lvl    AllPub     Corner        Gtl        NAmes   
3   NaN       Reg          Lvl    AllPub     Corner        Gtl        NAmes   
4   NaN       IR1          Lvl    AllPub     Inside        Gtl      Gilbert   

  Condition 1 Condition 2 Bldg Type House Style  Overall Qual  Overall Cond  \
0        Norm        Norm      1Fam      1Story             6             5   
1       Feedr        Norm      1Fam      1Story             5             6   
2        Norm        Norm      1Fam      1Story             6             6   
3        Norm        Norm      1Fam      1Story             7             5   
4        Norm        Norm      1Fam      2Story             5             5   

   Year Built  Year Remod/Add Roof Style Roof Matl Exterior 1st Exterior 2nd  \
0        1960            1960        Hip   CompShg      BrkFace      Plywood   
1        1961            1961      Gable   CompShg      VinylSd      VinylSd   
2        1958            1958        Hip   CompShg      Wd Sdng      Wd Sdng   
3        1968            1968        Hip   CompShg      BrkFace      BrkFace   
4        1997            1998      Gable   CompShg      VinylSd      VinylSd   

  Mas Vnr Type  Mas Vnr Area Exter Qual Exter Cond Foundation Bsmt Qual  \
0        Stone        112.00         TA         TA     CBlock        TA   
1          NaN          0.00         TA         TA     CBlock        TA   
2      BrkFace        108.00         TA         TA     CBlock        TA   
3          NaN          0.00         Gd         TA     CBlock        TA   
4          NaN          0.00         TA         TA      PConc        Gd   

  Bsmt Cond Bsmt Exposure BsmtFin Type 1  BsmtFin SF 1 BsmtFin Type 2  \
0        Gd            Gd            BLQ        639.00            Unf   
1        TA            No            Rec        468.00            LwQ   
2        TA            No            ALQ        923.00            Unf   
3        TA            No            ALQ      1,065.00            Unf   
4        TA            No            GLQ        791.00            Unf   

   BsmtFin SF 2  Bsmt Unf SF  Total Bsmt SF Heating Heating QC Central Air  \
0          0.00       441.00       1,080.00    GasA         Fa           Y   
1        144.00       270.00         882.00    GasA         TA           Y   
2          0.00       406.00       1,329.00    GasA         TA           Y   
3          0.00     1,045.00       2,110.00    GasA         Ex           Y   
4          0.00       137.00         928.00    GasA         Gd           Y   

  Electrical  1st Flr SF  2nd Flr SF  Low Qual Fin SF  Gr Liv Area  \
0      SBrkr        1656           0                0         1656   
1      SBrkr         896           0                0          896   
2      SBrkr        1329           0                0         1329   
3      SBrkr        2110           0                0         2110   
4      SBrkr         928         701                0         1629   

   Bsmt Full Bath  Bsmt Half Bath  Full Bath  Half Bath  Bedroom AbvGr  \
0            1.00            0.00          1          0              3   
1            0.00            0.00          1          0              2   
2            0.00            0.00          1          1              3   
3            1.00            0.00

In [4]:
# Para este taller, solo usaremos algunas columnas clave para simplificar.
df_sub = df[['Overall Qual', 'Gr Liv Area', 'SalePrice']].copy()
df_sub.columns = ['OverallQual', 'GrLivArea', 'SalePrice'] # Renombrar para facilidad
df_sub.head()

OverallQual  GrLivArea  SalePrice
0            6       1656     215000
1            5        896     105000
2            6       1329     172000
3            7       2110     244000
4            5       1629     189900

**Explicación de las variables del dataset reducido**

1. SalePrice: Precio de Venta.

Esta es la variable objetivo. Es el precio final por el cual se vendió la propiedad, medido en dólares estadounidenses.

2. OverallQual: Calidad General.

Es una variable ordinal que califica la calidad general del material y el acabado de la casa. Es una de las variables predictoras (X) más importantes.

Escala: Va de 1 a 10.

10: Muy Excelente

9: Excelente

...

2: Pobre

1: Muy Pobre

3. GrLivArea: Área Habitable sobre el Nivel del Suelo.

Es una variable numérica que mide el total de metros cuadrados de área habitable que está por encima del nivel del suelo. No incluye el área del sótano. Es una de las variables predictoras (X) más fuertes, ya que, lógicamente, casas más grandes tienden a ser más caras.

In [ ]:
df_sub.info()

### **Ejercicio 2: Dividir el conjunto de datos**

El método `.info()` muestra que no hay nulos en nuestro subconjunto. ¡Perfecto! Ahora, definamos nuestras variables `X` (predictoras) e `y` (objetivo) y dividamos los datos.

In [5]:
# Definir X e y
### TU CÓDIGO AQUÍ ###
X = df_sub[['OverallQual', 'GrLivArea']]
y = df_sub['SalePrice']

In [6]:
# Dividir en entrenamiento y prueba (70/30)
### TU CÓDIGO AQUÍ ###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Tamaño del set de Entrenamiento: {X_train.shape[0]} observaciones")
print(f"Tamaño del set de Prueba: {X_test.shape[0]} observaciones")

Tamaño del set de Entrenamiento: 2051 observaciones
Tamaño del set de Prueba: 879 observaciones


### **Ejercicio 3: Modelo con Sobreajuste**

Vamos a crear un modelo muy complejo para ver qué tan mal puede generalizar.

**Crear un Modelo Polinómico de Grado 5**

Usa `Pipeline` para combinar `PolynomialFeatures` (grado 5), `StandardScaler` y `LinearRegression`. Entrénalo con los datos de entrenamiento.

In [8]:
# Crear el pipeline para el modelo polinómico
### TU CÓDIGO AQUÍ ###
pipeline_poly = Pipeline([
    ("poly_features", PolynomialFeatures(degree=5, include_bias=False)),
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])


In [9]:
# Entrenar el modelo
### TU CÓDIGO AQUÍ ###
pipeline_poly.fit(X_train, y_train)

# Predicciones y error
y_train_pred_poly = pipeline_poly.predict(X_train)
y_test_pred_poly = pipeline_poly.predict(X_test)

In [11]:
# Calcular el error (RMSE) en entrenamiento y prueba y realizar un print de estos
### TU CÓDIGO AQUÍ ###
rmse_train_poly = np.sqrt(mean_squared_error(y_train, y_train_pred_poly))
rmse_test_poly = np.sqrt(mean_squared_error(y_test, y_test_pred_poly))

print(f"RMSE en Entrenamiento (Poli-5): ${rmse_train_poly:,.2f} ")
print(f"RMSE en Prueba (Poli-5): ${rmse_test_poly:,.2f}")

RMSE en Entrenamiento (Poli-5): $34,152.35 
RMSE en Prueba (Poli-5): $41,101.60


**Pregunta:** ¿Qué indica la diferencia que observas entre el error de entrenamiento y el de prueba? ¿Le recomendarías este modelo a la firma inmobiliaria? ¿Por qué?

El modelo Polinómico de grado 5 presenta un RMSE de $34,152.35 en entrenamiento y $41,101.60 en prueba, lo que evidencia un ligero sobreajuste.
Aunque el modelo logra capturar bien la relación entre las variables, su capacidad de generalización podría mejorar.
Por ello, no se recomienda como modelo final para la firma inmobiliaria sin realizar ajustes o comparar su desempeño con alternativas más simples y regulares.

### **Ejercicio 4: Aplicar Regularización**

Ahora, vamos a "curar" el sobreajuste. Usaremos los mismos `PolynomialFeatures` de grado 5, pero cambiaremos el modelo de regresión.

**Implementar Regresión Ridge**

Copia el pipeline anterior, pero reemplaza `LinearRegression` con `Ridge(alpha=10)`. `alpha` es la fuerza de la penalización.

In [16]:
# Crear el pipeline para Ridge
### TU CÓDIGO AQUÍ ###
pipeline_ridge = Pipeline([
    ("poly_features", PolynomialFeatures(degree=5, include_bias=False)),
    ("scaler", StandardScaler()),
    ("model", Ridge(alpha=10))
])

pipeline_ridge.fit(X_train, y_train)

Pipeline(steps=[('poly_features',
                 PolynomialFeatures(degree=5, include_bias=False)),
                ('scaler', StandardScaler()), ('model', Ridge(alpha=10))])

In [17]:
# Entrenar el modelo
### TU CÓDIGO AQUÍ ###
y_train_pred_ridge = pipeline_ridge.predict(X_train)
y_test_pred_ridge = pipeline_ridge.predict(X_test)

In [19]:
# Calcular el error (RMSE) en entrenamiento y prueba y realizar un print de estos
### TU CÓDIGO AQUÍ ###
rmse_train_ridge = np.sqrt(mean_squared_error(y_train, y_train_pred_ridge))
rmse_test_ridge = np.sqrt(mean_squared_error(y_test, y_test_pred_ridge))

print(f"RMSE Entrenamiento (Ridge): ${rmse_train_ridge:,.2f}")
print(f"RMSE Prueba (Poli-5): ${rmse_test_poly:,.2f} ")
print(f"RMSE Prueba (Ridge): ${rmse_test_ridge:,.2f} ")

RMSE Entrenamiento (Ridge): $35,204.89
RMSE Prueba (Poli-5): $41,101.60 
RMSE Prueba (Ridge): $35,206.63 


**Interpreta los resultados:**

**Implementar Regresión Lasso**

Haz lo mismo, pero ahora con `Lasso(alpha=500, max_iter=10000)`. Lasso necesita un `alpha` más grande (porque la penalización L1 es diferente) y a veces más `max_iter` para converger.

In [20]:
# Crear el pipeline para Lasso
### TU CÓDIGO AQUÍ ###
pipeline_lasso = Pipeline([
    ("poly_features", PolynomialFeatures(degree=5, include_bias=False)),
    ("scaler", StandardScaler()),
    ("model", Lasso(alpha=500, max_iter=10000))
])

pipeline_lasso.fit(X_train, y_train)

Pipeline(steps=[('poly_features',
                 PolynomialFeatures(degree=5, include_bias=False)),
                ('scaler', StandardScaler()),
                ('model', Lasso(alpha=500, max_iter=10000))])

In [21]:
# Entrenar el modelo
### TU CÓDIGO AQUÍ ###
y_train_pred_lasso = pipeline_lasso.predict(X_train)
y_test_pred_lasso = pipeline_lasso.predict(X_test)

In [23]:
# Calcular el error (RMSE) en entrenamiento y prueba y realizar un print de estos
### TU CÓDIGO AQUÍ ###
rmse_train_lasso = np.sqrt(mean_squared_error(y_train, y_train_pred_lasso))
rmse_test_lasso = np.sqrt(mean_squared_error(y_test, y_test_pred_lasso))

print(f"RMSE Entrenamiento (Lasso): ${rmse_train_lasso:,.2f}")
print(f"RMSE Prueba (Ridge): ${rmse_test_ridge:,.2f}")
print(f"RMSE Prueba (Lasso): ${rmse_test_lasso:,.2f}")

RMSE Entrenamiento (Lasso): $35,352.92
RMSE Prueba (Ridge): $35,206.63
RMSE Prueba (Lasso): $35,447.71


**Interpreta los resultados:**

**Selección de Variables con Lasso**

Una de las grandes ventajas de Lasso es que puede eliminar variables. Vamos a ver cuántas de las características polinómicas que creamos (ej. `OverallQual^2`, `GrLivArea^5`, `OverallQual * GrLivArea^4`, etc.) fueron eliminadas.

In [ ]:
# Extraer los coeficientes del modelo Lasso
### TU CÓDIGO AQUÍ ###


In [ ]:
# Extraer los nombres de las características generadas
# Guíate por el siguiente código: feature_names = pipeline_lasso.named_steps['poly_features'].get_feature_names_out(X.columns)
### TU CÓDIGO AQUÍ ###

In [ ]:
# Contar cuántos coeficientes son exactamente cero
### TU CÓDIGO AQUÍ ###


In [ ]:
# Realizar un print de:
# Número total de características polinómicas generadas
# Número de características eliminadas por Lasso (coef = 0)
# Número de características CONSERVADAS
# Porcentaje de variables eliminadas
# Lista con los nombres de las características conservadas por Lasso (coef != 0)

### TU CÓDIGO AQUÍ ###


**Interpreta los resultados:**

### **Ejercicio 5: Conclusión y Recomendación para el Cliente**

**Resumir los Resultados**

Crea un `DataFrame` de pandas que compare el RMSE de entrenamiento y prueba de los tres modelos (Polinómico, Ridge, Lasso) y ordena los modelos según el RMSE de prueba de menor a mayor.

In [ ]:
# Crear DataFrame de resultados
### TU CÓDIGO AQUÍ ###


**Pregunta Final:**

Basado en tu análisis, ¿qué modelo le recomendarías a la firma inmobiliaria? Tu respuesta debe incluir:

1.  Una recomendación clara del mejor modelo.
2.  Una explicación en términos sencillos (para un gerente, no para un científico de datos) de por qué el modelo polinómico simple no era una buena idea, usando el concepto de "memorizar vs. generalizar".
3.  Una descripción de la ventaja principal del modelo Lasso en cuanto a la interpretabilidad y la selección de las características más importantes.